In [ ]:
gnames = [f for f in os.listdir(crpath) if 'gray' in f.lower() and f.endswith('jpg')]
gpths = [ os.path.join(crpath,gn) for gn in gnames ] 
grays = [cv2.cvtColor(cv2.imread(g),cv2.COLOR_BGR2GRAY) for g in gpths]

for i,g in enumerate(grays):
    h, w = g.shape[:2]
    scale = 64 / min(h, w)
    resized = cv2.resize(g, (int(w * scale), int(h * scale)))
    fn = gnames[i]
    fn = fn.split('.jpg')[0]+'-DOWNSCALED.jpg'
    oPATH = os.path.join(crpath,fn)
    cv2.imwrite(oPATH,g)